In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SessionCentexETL") \
    .master("local[4]") \
    .config("spark.sql.shuffle.partitions", 4) \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars", "C:\\jars\\mssql-jdbc-12.10.0.jre11.jar") \
    .getOrCreate()

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

jdbc_url = f"jdbc:sqlserver://{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')};databaseName={os.getenv('DB_NAME')};encrypt=true;trustServerCertificate=true"
properties = {
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [30]:
query_pa = "(select * from Doc_Compra) AS query_pa"
df_empresas = spark.read.jdbc(url=jdbc_url, table=query_pa, properties=properties)
df_ruc = df_empresas.select("ID_Agenda").distinct().limit(60)

print(f"Total distinct 'ID_Agenda' in empresas: {df_ruc.count()}")

df_ruc.show(df_ruc.count(), truncate=False)

Total distinct 'ID_Agenda' in empresas: 60
+--------------+
|ID_Agenda     |
+--------------+
|03882991      |
|05343586      |
|06070544      |
|07247883000269|
|10004191604   |
|10085296391   |
|10096359701   |
|10252120      |
|10726397355   |
|15429137      |
|15982454      |
|18171528      |
|20100041953   |
|20100049181   |
|20100050359   |
|20100365341   |
|20101158927   |
|20101520898   |
|20108811375   |
|20119546851   |
|20122621830   |
|20127765279   |
|20131312955   |
|20251549835   |
|20380280907   |
|20381235051   |
|20390900407   |
|20404698428   |
|20448661378   |
|20455823880   |
|20469653707   |
|20471195881   |
|20481828792   |
|20502007018   |
|20505018509   |
|20507379002   |
|20507661441   |
|20520543466   |
|20523634110   |
|20536124404   |
|20536563823   |
|20537746956   |
|20539528441   |
|20539666917   |
|20543602664   |
|20544466080   |
|20544488059   |
|20547025319   |
|20547254776   |
|20548704261   |
|20549068201   |
|20550372640   |
|20550688442   |
|2055

In [21]:
import requests

def consulta_api(ruc):
    url = f"http://127.0.0.1:8000/v2/sunat/ruc/{ruc}"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Status {response.status_code}"}
    except Exception as e:
        return {"error": str(e)}

resultados = [consulta_api(row['ID_Agenda']) for row in df_ruc.collect()]

In [26]:
from pyspark.sql.types import StructType, StructField, StringType

campos_resultados = ["razonSocial", "tipoDocumento", "numeroDocumento", "estado", "condicion", "direccion", "ubigeo", "viaTipo", "viaNombre"]

schema = StructType([StructField(campo, StringType(), True) for campo in campos_resultados])

resultados_limpios = [
    {campo: resultado.get(campo, None) for campo in campos_resultados}
    for resultado in resultados if isinstance(resultado, dict)
]

print("Cantidad de filas en resultados_limpios:", len(resultados_limpios))
if len(resultados_limpios) == 0:
    print("¡La lista está vacía! No se puede crear un DataFrame vacío en Spark con esquema inferido.")

print("Ejemplo de resultados crudos:")
for r in resultados_limpios:
    print(r)

#resultados_limpios = [r for r in resultados_limpios if any(v is not None for v in r.values())]

#df_ruc_resultados = spark.createDataFrame(resultados_limpios, schema=schema)
#df_ruc_resultados.coalesce(1).write.mode("overwrite").option("header", True).csv("DevueltosCSV/")

Cantidad de filas en resultados_limpios: 10
Ejemplo de resultados crudos:
{'razonSocial': 'SOUTHERN PERU COPPER CORPORATION, SUCURSAL DEL PERÚ', 'tipoDocumento': '6', 'numeroDocumento': '20100147514', 'estado': 'ACTIVO', 'condicion': 'HABIDO', 'direccion': 'AV. CAMINOS DEL INCA NRO 171 URB. CHACARILLA DEL ESTANQUE ', 'ubigeo': '150140', 'viaTipo': 'AV.', 'viaNombre': 'CAMINOS DEL INCA'}
{'razonSocial': 'COMPAÑIA MINERA ANTAPACCAY S.A.', 'tipoDocumento': '6', 'numeroDocumento': '20114915026', 'estado': 'ACTIVO', 'condicion': 'HABIDO', 'direccion': 'CAMPAMENTO MINERO TINTAYA NRO SN CAMPAMENTO MINERO TINTAYA ', 'ubigeo': '080801', 'viaTipo': '-', 'viaNombre': 'CAMPAMENTO MINERO TINTAYA'}
{'razonSocial': 'CAL & CEMENTO SUR S.A.', 'tipoDocumento': '6', 'numeroDocumento': '20115039262', 'estado': 'ACTIVO', 'condicion': 'HABIDO', 'direccion': 'CAR. JULIACA-PUNO HACIENDA YUNGURA KM. 11 ', 'ubigeo': '211104', 'viaTipo': 'CAR.', 'viaNombre': 'JULIACA-PUNO'}
{'razonSocial': 'SUPERINTENDENCIA NACI

In [27]:
import pandas as pd
df_pd = pd.DataFrame(resultados_limpios)

df_pd.to_csv("DevueltosCSV/resultados.csv", index=False, encoding='utf-8-sig')